<a href="https://colab.research.google.com/github/kaushalprasadhial/face-recognition/blob/master/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def movefile(source,dest):
    os.rename(source, dest)

In [2]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 3.5MB/s 
  Created wheel for mtcnn: filename=mtcnn-0.0.9-cp36-none-any.whl size=2257690 sha256=6ac45254fd5a00c549cc3891b31f8d78f0fc2b72ece134782bd73f6539324e64
  Stored in directory: /root/.cache/pip/wheels/85/81/65/6363fa5aafd7a155c896591e0c7c6e27b69642aa82b9cbf076
Successfully built mtcnn


In [3]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-5hl15eul
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-5hl15eul
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=eb9a925704fec12094ebcf1bc707426336a43fdddc01a6655a0d2fb7d7f28d8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-f0tjkcb_/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [4]:
import numpy as np
import cv2
from keras.models import Model, Sequential, model_from_json
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop
from keras.models import load_model

from mtcnn.mtcnn import MTCNN

import matplotlib.pyplot as plt

import os
from os import walk
from os import listdir
from os.path import join
import glob

import cv2

Using TensorFlow backend.


In [5]:
detector = MTCNN()







Instructions for updating:
Deprecated in favor of operator or tf.math.divide.



In [0]:
batch_size = 20
epochs = 2
steps_per_epoch = 100

In [7]:
train_dir = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train'
test_dir = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test'

input_shape=(224,224)
datagen_train = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      vertical_flip=True,
      fill_mode='nearest')
datagen_test = ImageDataGenerator(rescale=1./255)
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)
generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

steps_test = generator_test.n / batch_size
num_classes = generator_train.num_classes
cls_train = generator_train.classes
cls_test = generator_test.classes

Found 487 images belonging to 6 classes.
Found 147 images belonging to 6 classes.


In [0]:
def path_join(dirname, filenames):
    return [os.path.join(dirname, filename) for filename in filenames]

In [9]:
steps_test = generator_test.n / batch_size
image_paths_train = path_join(train_dir, generator_train.filenames)
image_paths_test = path_join(test_dir, generator_test.filenames)
cls_train = generator_train.classes
cls_test = generator_test.classes
class_names = list(generator_train.class_indices.keys())
print('class name',class_names)
num_classes = generator_train.num_classes
print('num classes',num_classes)

class name ['Scarlett Johansson', 'Willa Holland', 'alexandra daddario', 'amber heard face', 'anne hathaway', 'kaushal']
num classes 6


In [82]:
def loadVggFaceModel():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    model.load_weights('/content/drive/My Drive/Colab Notebooks/face recognition/vgg_face_weights.h5')
    loaded_model = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)
#     predictions = Dense(6, activation = 'softmax')(loaded_model.output)
#     new_model = Model(input = loaded_model.input, output = predictions)
    return loaded_model

vgg_model = loadVggFaceModel()
vgg_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_14_input (Inp (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_14 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 114, 114, 64)      0   

In [0]:
new_model = Sequential()
new_model.add(vgg_model)
new_model.add(Dense(1024, activation='relu'))
new_model.add(Dropout(.5))
new_model.add(Dense(num_classes,activation='softmax'))

In [0]:
optimizer = Adam(lr=1e-5)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']

In [85]:
def print_layer_trainable(model):
    for layer in model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))
        
print_layer_trainable(vgg_model)

False:	zero_padding2d_14_input
True:	zero_padding2d_14
True:	conv2d_17
True:	zero_padding2d_15
True:	conv2d_18
True:	max_pooling2d_6
True:	zero_padding2d_16
True:	conv2d_19
True:	zero_padding2d_17
True:	conv2d_20
True:	max_pooling2d_7
True:	zero_padding2d_18
True:	conv2d_21
True:	zero_padding2d_19
True:	conv2d_22
True:	zero_padding2d_20
True:	conv2d_23
True:	max_pooling2d_8
True:	zero_padding2d_21
True:	conv2d_24
True:	zero_padding2d_22
True:	conv2d_25
True:	zero_padding2d_23
True:	conv2d_26
True:	max_pooling2d_9
True:	zero_padding2d_24
True:	conv2d_27
True:	zero_padding2d_25
True:	conv2d_28
True:	zero_padding2d_26
True:	conv2d_29
True:	max_pooling2d_10
True:	conv2d_30
True:	dropout_4
True:	conv2d_31
True:	dropout_5
True:	conv2d_32
True:	flatten_2


In [86]:
vgg_model.trainable = False #i dont know why it didnt work
for layer in vgg_model.layers:
    layer.trainable = False
print_layer_trainable(new_model)
print('new model')
print_layer_trainable(vgg_model)

False:	model_2
True:	dense_3
True:	dropout_6
True:	dense_4
new model
False:	zero_padding2d_14_input
False:	zero_padding2d_14
False:	conv2d_17
False:	zero_padding2d_15
False:	conv2d_18
False:	max_pooling2d_6
False:	zero_padding2d_16
False:	conv2d_19
False:	zero_padding2d_17
False:	conv2d_20
False:	max_pooling2d_7
False:	zero_padding2d_18
False:	conv2d_21
False:	zero_padding2d_19
False:	conv2d_22
False:	zero_padding2d_20
False:	conv2d_23
False:	max_pooling2d_8
False:	zero_padding2d_21
False:	conv2d_24
False:	zero_padding2d_22
False:	conv2d_25
False:	zero_padding2d_23
False:	conv2d_26
False:	max_pooling2d_9
False:	zero_padding2d_24
False:	conv2d_27
False:	zero_padding2d_25
False:	conv2d_28
False:	zero_padding2d_26
False:	conv2d_29
False:	max_pooling2d_10
False:	conv2d_30
False:	dropout_4
False:	conv2d_31
False:	dropout_5
False:	conv2d_32
False:	flatten_2


In [0]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [10]:
from sklearn.utils.class_weight import compute_class_weight
class_weight = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(cls_train),
                                    y=cls_train)
print(class_weight)

[0.86347518 1.22979798 0.75154321 1.1763285  0.9898374  1.19362745]


In [0]:
#fit new vgg model
all_history =[]
for i in range(100):
    history = new_model.fit_generator(
            generator=generator_train,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            class_weight=class_weight,
            validation_data=generator_test)
    generator_test.reset()
    result = loaded_vgg_model.evaluate_generator(generator_test, steps=steps_test)
    print('result: ',result)
    if i%10==0:
        new_model.save('/content/drive/My Drive/Colab Notebooks/face recognition/model.h5')

Epoch 1/2
100/100 [==============================] - 41s 408ms/step - loss: 1.7766 - categorical_accuracy: 0.2416 - val_loss: 1.7369 - val_categorical_accuracy: 0.6122
Epoch 2/2
100/100 [==============================] - 39s 392ms/step - loss: 1.7452 - categorical_accuracy: 0.3957 - val_loss: 1.6892 - val_categorical_accuracy: 0.6122
result:  [1.1913758240589478, 0.8027210819477938]
Epoch 1/2
100/100 [==============================] - 41s 412ms/step - loss: 1.7133 - categorical_accuracy: 0.4281 - val_loss: 1.6382 - val_categorical_accuracy: 0.5986
Epoch 2/2
100/100 [==============================] - 39s 390ms/step - loss: 1.6795 - categorical_accuracy: 0.4496 - val_loss: 1.5855 - val_categorical_accuracy: 0.6190
result:  [1.1913758240589478, 0.8027210819477938]
Epoch 1/2
100/100 [==============================] - 41s 407ms/step - loss: 1.6457 - categorical_accuracy: 0.4643 - val_loss: 1.5339 - val_categorical_accuracy: 0.6190
Epoch 2/2
100/100 [==============================] - 39s 390

In [11]:
print('loading model')
loaded_vgg_model = load_model('/content/drive/My Drive/Colab Notebooks/face recognition/model.h5')
print('model loaded')

loading model



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


model loaded


In [12]:
loaded_vgg_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 2622)              145002878 
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              2685952   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 6150      
Total params: 147,694,980
Trainable params: 2,692,102
Non-trainable params: 145,002,878
_________________________________________________________________


In [0]:
print_layer_trainable(loaded_vgg_model)
for layer in loaded_vgg_model.layers:
    layer.trainable = False
for layer in loaded_vgg_model.layers[-17:]:
    layer.trainable = True
print('after unfreazing')
print_layer_trainable(loaded_vgg_model)

In [0]:
optimizer = Adam(lr=1e-5)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']
loaded_vgg_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [0]:
all_history =[]
count =0
for i in range(100):
    history=loaded_vgg_model.fit_generator(
            generator=generator_train,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            class_weight=class_weight,
            validation_data=generator_test)
    all_history.append(history)
    generator_test.reset()
    result = loaded_vgg_model.evaluate_generator(generator_test, steps=steps_test)
    print('result: ',result)
    if i%10==0:
        loaded_vgg_model.save('/content/drive/My Drive/Colab Notebooks/face recognition/tuned_model.h5')

In [73]:
cap = cv2.VideoCapture(/content/drive/My Drive/Colab Notebooks/face recognition/20191014_180017.mp4) 

while(True):
    ret, img = cap.read()
    #img = cv2.imread(img)
    print(img.shape())
    faces = detector.detect_faces(img)
    for (x,y,w,h) in faces:
        if w > 130: 
            #cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) #draw rectangle to main image

            detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
            detected_face = cv2.resize(detected_face, (224, 224)) #resize to 224x224

            img_pixels = image.img_to_array(detected_face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)

            img_pixels /= 127.5
            img_pixels -= 1

            captured_representation = loaded_vgg_model.predict(img_pixels)

            print('captured_representation ',captured_representation)
            predicted_person = captured_representation.index(max(captured_representation))
            print('predicted_person ',predicted_person)
            confidence = max(captured_representation)
            print('confidence ',confidence)
            #y_prob = model.predict(x) 
            y_classes = captured_representation.argmax(axis=-1)
            print(y_classes)
            #text = class_names[predicted_person]

            if confidence>.6:
                text = class_names[predicted_person]
            else:
                text = 'unknown'
            cv2.putText(img, text, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

            cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),color,1)
            cv2.line(img,(x+w,y-20),(x+w+10,y-20),color,1)

        # 			if(found == 0): #if found image is not in employee database
        # 				cv2.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        #             else:
        #                 cv2.putText(img, person_name, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

        cv2.imshow('img',img)

    if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
        break

	
#kill open cv things		
cap.release()
cv2.destroyAllWindows()

AttributeError: ignored